In [1]:
# Standard imports for bokeh visualization package
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [72]:
############################################# Making dummy data #################################################
# ***Important note***: for actual data-taking, this cell will be obsolete, because the measured data will be read in via a csv file.
import numpy as np

# Make arrays of dummy data for spray duration, spray frequency, and basket configuration.
#### For the dummy data, five basket configurations are tested, labelled "config_#"".
spray_duration_s = np.linspace(1, 10, 10)
spray_interval_s = np.linspace(10, 60, 10)
basket_configs = ['config_1', 'config_2', 'config_3', 'config_4', 'config_5']
basket_config_indices = 1 + np.arange(len(basket_configs))

# Make a 2D mesh of the durations and spray intervals that covers all combinations of spray intervals and durations
spray_duration_s_mesh, spray_interval_s_mesh = np.meshgrid(spray_duration_s, spray_interval_s) 

# Flatten the 2D arrays of spray duration and interval to make 1D arrays with all combinations
spray_duration_s = spray_duration_s_mesh.ravel()
spray_interval_s = spray_interval_s_mesh.ravel()

# Add some jitter to randomize the duration and and intervals a bit
spray_duration_s = spray_duration_s + np.random.rand(len(spray_duration_s)) * 0.1 * ( max(spray_duration_s) - min(spray_duration_s) )
spray_interval_s = spray_interval_s + np.random.rand(len(spray_interval_s)) * 0.1 * ( max(spray_interval_s) - min(spray_interval_s) )

###### Construct dummy data for the CO2 ppm as 2D Gaussian functions of spray interval and spray duration. ######
# For each basket configuration, set a different peak location of the Gaussian function in spray duration and interval 
spray_duration_peaks = min(spray_duration_s) + basket_config_indices * \
                       ( max(spray_duration_s) - min(spray_duration_s) ) / (len(basket_config_indices) + 1.)
spray_interval_peaks = min(spray_interval_s) + basket_config_indices * \
                       ( max(spray_interval_s) - min(spray_interval_s) ) / (len(basket_config_indices) + 1.)

# Set the widths of the dummy spray duration and interval Gaussian peaks to 1/4 their total measured range
spray_duration_width = ( max(spray_duration_s) - min(spray_duration_s) ) / 4.
spray_interval_width = ( max(spray_interval_s) - min(spray_interval_s) ) / 4.

# Make the dummy 2D Gaussian functions for each configuration
CO2_ppms = {}
for i in range(len(basket_configs)):
    CO2_ppms[basket_configs[i]] = np.exp(-( \
                                            np.power(spray_duration_s - spray_duration_peaks[i], 2.) \
                                            / (2 * np.power(spray_duration_width, 2.)) + \
                                            np.power(spray_interval_s - spray_interval_peaks[i], 2.) \
                                            / (2 * np.power(spray_interval_width, 2.)) \
                                          ) \
                                        )

In [74]:
# This cell uses the bokeh package to visualize the CO2 ppm as a function of spray frequency and 

from bokeh.plotting import figure
from bokeh.models import LinearColorMapper
from bokeh.models import ColumnDataSource
from bokeh.layouts import column

iconfig=0
config = basket_configs[iconfig]
source = ColumnDataSource(data = dict(duration = spray_duration_s, interval = spray_interval_s, colour = CO2_ppms[config]))
exp_cmap = LinearColorMapper(palette="Viridis256", 
                             low = min(CO2_ppms[config]), 
                             high = max(CO2_ppms[config]))

p = figure(width=400, height=400, title="CO2 concentration (ppm)")
p.circle('duration', 'interval', size=10, source=source, line_color=None,
        fill_color={"field":"colour", "transform":exp_cmap})
p.xaxis.axis_label = 'Spray Duration (s)'
p.yaxis.axis_label = 'Spray Interval (s)'

from bokeh.models import ColorBar
bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
p.add_layout(bar, "left")

from bokeh.models import CustomJS, RadioButtonGroup
radio_button_group = RadioButtonGroup(labels=basket_configs, active=0)
#radio_button_group.on_change('active', lambda attr, old, new: update())

#def radiogroup_click(attr,old,new):
#    active_radio=radio_button_group.active ##Getting radio button value

#    # filter the dataframe with value in radio-button
#    config = basket_configs[active_radio]
#    source.data = {'duration':spray_duration_s,'interval':spray_interval_s,'colour':CO2_ppms[config]}

#radio_button_group.on_change('active',radiogroup_click)
update_radio = CustomJS(args=dict(basket_configs=basket_configs, CO2_ppms=CO2_ppms, source=source), code="""
    console.log('radio_button_group: active=' + this.active)
    const duration = source.data.duration
    const interval = source.data.interval
    const colour = CO2_ppms[basket_configs[this.active]]
    source.data = { duration, interval, colour }
    
""")

radio_button_group.js_on_click(update_radio)

show(column(radio_button_group, p))

In [ ]:
# In this cell, I want to visualize the CO2 ppm as a function of either spray duration or frequency.
# In each case, it will be profiled to maximize over the variable that isn't displayed. 